### Set up

In [3]:
import geopandas as gpd
import pandas as pd

In [6]:
path_in = "/Users/sofia/Documents/Repos/skytruth_30x30/data/raw"
path_out = "/Users/sofia/Documents/Repos/skytruth_30x30/data/processed/mpatlas"

### MPAtlas

In [7]:
# Read data
mpatlas = gpd.read_file(path_in + "/MPAtlas_largest100.shp")

In [8]:
mpatlas.columns

Index(['OBJECTID', 'WDPAID', 'WDPA_PID', 'NAME', 'English_De', 'PARENT_ISO',
       'ISO3', 'MPA_Marine', 'mpa_id', 'Zone_Marin', 'IUCN_Cat', 'Stage_of_E',
       'Distant_MP', 'Level_of_P', 'Most_Impac', 'Descrip_Im', 'Vertically',
       'SHAPE_Leng', 'SHAPE_Area', 'geometry'],
      dtype='object')

In [12]:
mpatlas.Level_of_P.unique()

array(['Incompatible', 'Highly', 'TBD', 'Fully', 'Lightly', 'Unknown'],
      dtype=object)

In [8]:
# Create new column with protection level reclassified
def map_protection_level(value):
    if value in ["Fully", "Highly"]:
        return "Fully / Highly Protected"
    else:
        return "Less Protected / Unknown"

# Create a new column based on column1
mpatlas['P_LEVEL'] = mpatlas['Level_of_P'].apply(map_protection_level)
mpatlas.head(2)

,OBJECTID,WDPAID,WDPA_PID,NAME,English_De,PARENT_ISO,ISO3,MPA_Marine,mpa_id,Zone_Marin,...,Stage_of_E,Distant_MP,Level_of_P,Most_Impac,Descrip_Im,Vertically,SHAPE_Leng,SHAPE_Area,geometry,P_LEVEL
0,1.0,478053.0,478053,Hikurangi Deep,Benthic Protection Area,NZL,NZL,54022.1,5258,54022.1,...,Implemented,NaN,Incompatible,"Mining, Fishing",Benthic protections only. Deep sea mining allo...,X,12.332952,5.833001,"POLYGON ((-175.00000 -42.16661, -175.00000 -42...",Less Protected / Unknown
1,2.0,555512062.0,555512062,Kermadec,Benthic Protection Area,NZL,NZL,619146.0,5428,458540.5,...,Implemented,NaN,Incompatible,"Mining, Fishing",Benthic protections only. Deep sea mining allo...,NaN,25.629352,42.963159,"POLYGON ((-174.02370 -29.22191, -174.02370 -29...",Less Protected / Unknown


In [9]:
# List of dictionaries for data in Region_ISO3_PP.txt (list of regions used in the Protected Planet database)
regions_data = [
    {
        'region_iso': 'AS',
        'region_name': 'Asia & Pacific',
        'country_iso_3s': [
            "AFG", "ASM", "AUS", "BGD", "BRN", "BTN", "CCK", "CHN", "COK", "CXR", "FJI", "FSM", "GUM", "HKG", "IDN",
            "IND", "IOT", "IRN", "JPN", "KHM", "KIR", "KOR", "LAO", "LKA", "MAC", "MDV", "MHL", "MMR", "MNG", "MNP",
            "MYS", "NCL", "NFK", "NIU", "NPL", "NRU", "NZL", "PAK", "PCN", "PHL", "PLW", "PNG", "PRK", "PYF", "SGP",
            "SLB", "THA", "TKL", "TLS", "TON", "TUV", "TWN", "VNM", "VUT", "WLF", "WSM"
        ]
    },
    {
        'region_iso': 'AF',
        'region_name': 'Africa',
        'country_iso_3s': [
            "AGO", "BDI", "BEN", "BFA", "BWA", "CAF", "CIV", "CMR", "COD", "COG", "COM", "CPV", "DJI", "DZA", "EGY",
            "ERI", "ESH", "ETH", "GAB", "GHA", "GIN", "GMB", "GNB", "GNQ", "KEN", "LBR", "LBY", "LSO", "MAR", "MDG",
            "MLI", "MOZ", "MRT", "MUS", "MWI", "MYT", "NAM", "NER", "NGA", "REU", "RWA", "SDN", "SEN", "SHN", "SLE",
            "SOM", "SSD", "STP", "SWZ", "SYC", "TCD", "TGO", "TUN", "TZA", "UGA", "ZAF", "ZMB", "ZWE"
        ]
    },
    {
        'region_iso': 'EU',
        'region_name': 'Europe',
        'country_iso_3s': [
            "ALA", "ALB", "AND", "ARM", "AUT", "AZE", "BEL", "BGR", "BIH", "BLR", "CHE", "CYP", "CZE", "DEU", "DNK",
            "ESP", "EST", "FIN", "FRA", "FRO", "GBR", "GEO", "GGY", "GIB", "GRC", "HRV", "HUN", "IMN", "IRL", "ISL",
            "ISR", "ITA", "JEY", "KAZ", "KGZ", "LIE", "LTU", "LUX", "LVA", "MCO", "MDA", "MKD", "MLT", "MNE", "NLD",
            "NOR", "POL", "PRT", "ROU", "RUS", "SJM", "SMR", "SRB", "SVK", "SVN", "SWE", "TJK", "TKM", "TUR", "UKR",
            "UZB", "VAT"
        ]
    },
    {
        'region_iso': 'SA',
        'region_name': 'Latin America & Caribbean',
        'country_iso_3s': [
            "ABW", "AIA", "ARG", "ATG", "BES", "BHS", "BLM", "BLZ", "BMU", "BOL", "BRA", "BRB", "CHL", "COL", "CRI",
            "CUB", "CUW", "CYM", "DMA", "DOM", "ECU", "FLK", "GLP", "GRD", "GTM", "GUF", "GUY", "HND", "HTI", "JAM",
            "KNA", "LCA", "MAF", "MEX", "MSR", "MTQ", "NIC", "PAN", "PER", "PRI", "PRY", "SLV", "SUR", "SXM", "TCA",
            "TTO", "UMI", "URY", "VCT", "VEN", "VGB", "VIR"
        ]
    },
    {
        'region_iso': 'PO',
        'region_name': 'Polar',
        'country_iso_3s': [
            "ATF", "BVT", "GRL", "HMD", "SGS"
        ]
    },
    {
        'region_iso': 'NA',
        'region_name': 'North America',
        'country_iso_3s': [
            "CAN", "SPM", "USA"
        ]
    },
    {
        'region_iso': 'GL',
        'region_name': 'Global',
        'country_iso_3s': []
    },
    {
        'region_iso': 'WA',
        'region_name': 'West Asia',
        'country_iso_3s': [
            "ARE", "BHR", "IRQ", "JOR", "KWT", "LBN", "OMN", "PSE", "QAT", "SAU", "SYR", "YEM"
        ]
    }
]

# Convert the region data to a dictionary that maps each country to its region name
country_to_region = {}
for region in regions_data:
    for country in region['country_iso_3s']:
        country_to_region[country] = region['region_name']

In [10]:
mpatlas['REGIONS'] = mpatlas['ISO3'].map(country_to_region)
mpatlas.head(2)

,OBJECTID,WDPAID,WDPA_PID,NAME,English_De,PARENT_ISO,ISO3,MPA_Marine,mpa_id,Zone_Marin,...,Distant_MP,Level_of_P,Most_Impac,Descrip_Im,Vertically,SHAPE_Leng,SHAPE_Area,geometry,P_LEVEL,REGIONS
0,1.0,478053.0,478053,Hikurangi Deep,Benthic Protection Area,NZL,NZL,54022.1,5258,54022.1,...,NaN,Incompatible,"Mining, Fishing",Benthic protections only. Deep sea mining allo...,X,12.332952,5.833001,"POLYGON ((-175.00000 -42.16661, -175.00000 -42...",Less Protected / Unknown,Asia & Pacific
1,2.0,555512062.0,555512062,Kermadec,Benthic Protection Area,NZL,NZL,619146.0,5428,458540.5,...,NaN,Incompatible,"Mining, Fishing",Benthic protections only. Deep sea mining allo...,NaN,25.629352,42.963159,"POLYGON ((-174.02370 -29.22191, -174.02370 -29...",Less Protected / Unknown,Asia & Pacific


In [11]:
# Rename columns and keep only relevant ones. 
# Note: We keep "Zone_Marine" (area of the geometry), instead of "MPA_Marine" (as MPAs can be divided in smaller pieces according to their protection levels)

mpatlas = mpatlas.rename(columns={'English_De': 'DESIG_ENG', 'Zone_Marin': 'AREA_MPATLAS', 'Stage_of_E': 'ESTABLISHMENT', 'Most_Impac': 'IMPACT' }) 
mpatlas2 = mpatlas[['WDPAID', 'WDPA_PID', 'NAME', 'AREA_MPATLAS', 'DESIG_ENG', 'ESTABLISHMENT', 'IMPACT', 'P_LEVEL', 'PARENT_ISO', 'ISO3','REGIONS', 'geometry']]
mpatlas2.head(2)

,WDPAID,WDPA_PID,NAME,AREA_MPATLAS,DESIG_ENG,ESTABLISHMENT,IMPACT,P_LEVEL,PARENT_ISO,ISO3,REGIONS,geometry
0,478053.0,478053,Hikurangi Deep,54022.1,Benthic Protection Area,Implemented,"Mining, Fishing",Less Protected / Unknown,NZL,NZL,Asia & Pacific,"POLYGON ((-175.00000 -42.16661, -175.00000 -42..."
1,555512062.0,555512062,Kermadec,458540.5,Benthic Protection Area,Implemented,"Mining, Fishing",Less Protected / Unknown,NZL,NZL,Asia & Pacific,"POLYGON ((-174.02370 -29.22191, -174.02370 -29..."


In [12]:
mpatlas2.to_file(path_out + "/mpatlas_table.shp")

/var/folders/98/0pdnjc5s29x2pnzl293pw7hr0000gn/T/ipykernel_25742/67511564.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  mpatlas2.to_file(path_out + "/mpatlas_table.shp")


In [13]:
mpatlas = gpd.read_file(path_out + "/mpatlas_table.shp")
mpatlas.head(2)

,WDPAID,WDPA_PID,NAME,AREA_MPATL,DESIG_ENG,ESTABLISHM,IMPACT,P_LEVEL,PARENT_ISO,ISO3,REGIONS,geometry
0,478053.0,478053,Hikurangi Deep,54022.1,Benthic Protection Area,Implemented,"Mining, Fishing",Less Protected / Unknown,NZL,NZL,Asia & Pacific,"POLYGON ((-175.00000 -42.16661, -175.00000 -42..."
1,555512062.0,555512062,Kermadec,458540.5,Benthic Protection Area,Implemented,"Mining, Fishing",Less Protected / Unknown,NZL,NZL,Asia & Pacific,"POLYGON ((-174.02370 -29.22191, -174.02370 -29..."
